<a href="https://colab.research.google.com/github/VanshGarg123/Dynamic-Pricing-Model/blob/main/Copy_of_Dynamic_Pricing_Model(copy).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [ ]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [ ]:
df = pd.read_csv('dataset.csv')
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


#PreProcessing the Data

In [ ]:
# Combine date and time into a single Timestamp column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],format='%d-%m-%Y %H:%M:%S')

# Traffic level mapping
traffic_map = {"low": 1, "average": 2, "high": 3}
df["TrafficLevel"] = df["TrafficConditionNearby"].map(traffic_map)

# Vehicle type mapping (adjust weights as you like)
vehicle_weight_map = {"car": 1.0, "bike": 0.8, "truck": 1.5, "cycle": 0.5}
df["VehicleTypeWeight"] = df["VehicleType"].map(vehicle_weight_map)

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
# Save required columns
df[["Timestamp", "Occupancy", "Capacity", "QueueLength", "TrafficLevel", "IsSpecialDay", "VehicleTypeWeight"]].to_csv("parking_stream.csv", index=False)

In [ ]:
#Define Schema for Pathway
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficLevel: int
    IsSpecialDay: int
    VehicleTypeWeight: float

In [ ]:
#Simulate real-time stream
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

# Parse timestamp and get day bucket
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

#Model Pricing Function

In [ ]:
# Define weights and base price
α = 1.0
β = 0.5
γ = 0.7
δ = 1.0
ε = 0.6
λ = 0.8
base_price = 10.0

# Step 0: Windowing and Aggregation (per day)
delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        capacity=pw.reducers.max(pw.this.Capacity),
        queue=pw.reducers.max(pw.this.QueueLength),
        traffic=pw.reducers.max(pw.this.TrafficLevel),
        special=pw.reducers.max(pw.this.IsSpecialDay),
        vehicle_sum=pw.reducers.sum(pw.this.VehicleTypeWeight),
        vehicle_count=pw.reducers.count(pw.this.VehicleTypeWeight),

    )
)

In [ ]:
#For Model-1
model1 = delta_window.with_columns (
    price=10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.capacity
)

# Final price is computed and stored in model1

In [ ]:
#For Model-2

# Step 1: Compute mean vehicle weight
step1 = delta_window.with_columns(
    vehicle=pw.this.vehicle_sum / pw.this.vehicle_count
)

# Step 2: Compute raw demand
step2 = step1.with_columns(
    demand=α * ((pw.this.occ_max - pw.this.occ_min) / pw.this.capacity)
          + β * pw.this.queue
          - γ * pw.this.traffic
          + δ * pw.this.special
          + ε * pw.this.vehicle,
)

# Step 3: Compute global min/max demand
demand_minmax = step2.reduce(
        min_demand=pw.reducers.min(pw.this.demand),
        max_demand=pw.reducers.max(pw.this.demand),
)

# Add a dummy key column for join
step2_with_key = step2.with_columns(dummy_key=1)
demand_minmax_with_key = demand_minmax.with_columns(dummy_key=1)

# Join to add global min/max demand to each row
step3 = step2_with_key.join(
    demand_minmax_with_key,
    step2_with_key.dummy_key == demand_minmax_with_key.dummy_key,
)


# Step 5: Normalize demand
step4 = step3.select(pw.this.t,
    norm_demand=(
        pw.this.demand - pw.this.min_demand
    ) / (
        pw.this.max_demand-pw.this.min_demand + 1e-6
    )
)

# Step 6: Compute raw price
step5 = step4.with_columns(
    raw_price=base_price * (1 + λ * pw.this.norm_demand)
)

# Step 7: Clamp price to 0.5x–2x base_price
model2 = step5.with_columns(
    price=pw.if_else(
        pw.this.raw_price > 2 * base_price, 2 * base_price,
        pw.if_else(
            pw.this.raw_price < 0.5 * base_price, 0.5 * base_price,
            pw.this.raw_price
        )
    )
)

# Final price is computed and stored in model2

#Visualization

In [ ]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Dynamic Pricing Model-1",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order

# Plot first model
viz = model1.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Dynamic Pricing Model-2",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order

# Plot second model
viz = model2.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

Output()